# [2017-10-13] Get started with gensim

Comenzamos por parsear los tweets encontrados en `general-tweets-test-tagged.xml` y extraer de ellos solamente el contenido. Cada tweet corresponderá a una línea del archivo `tweets.txt`.

In [1]:
import xml.etree.ElementTree as ET

tweets = ET.parse('data/general-tweets-test-tagged.xml').getroot()

with open("data/tweets.txt", "a+") as f:
	for tweet in tweets:
		content = tweet.findall('content')[0].text
		# polarity = tweet.findall('sentiments')[0][0][0].text  # No extraemos esta información por ahora
		f.write("{}\n".format(content))

Con esto obtenemos un archivo en el siguiente formato:

```
Más de mañana en Gaceta. UPyD contará casi seguro con grupo gracias al Foro Asturias. Eso se dice en el Congreso
Muchas gracias a todos por los comentarios durante el programa de hoy en #VeoTV @MundoaCuestas :-))
Habia prometido responder a todos, pero me ha sido imposible. Y hoy no doy para mas. MUCHAS GRACIAS A TODOS
"Soy de ese tipo de personas que no acaban de comprender las cosas hasta que las ponen por escrito" Murakami
...
```

Para comenzar trabajando con `gensim` importamos `corpora` que nos permitirá generar un diccionario a partir de los tweets parseados.

In [2]:
from gensim import corpora

El archivo de tweets es manejable en memoria por lo que lo cargaremos en una lista de tweets. Consideraremos un documento por tweet.

In [3]:
with open("data/tweets.txt", "r") as f:
    documents = [line.strip() for line in f.readlines()]

Para preprocesar los documentos, comenzaremos por procesar las stopwords. Obviamente necesitamos stopwords en español y se han obtenido de https://github.com/6/stopwords-json.

In [4]:
stoplist = ["a","actualmente","acuerdo","adelante","ademas","además","adrede","afirmó","agregó","ahi","ahora","ahí","al","algo","alguna","algunas","alguno","algunos","algún","alli","allí","alrededor","ambos","ampleamos","antano","antaño","ante","anterior","antes","apenas","aproximadamente","aquel","aquella","aquellas","aquello","aquellos","aqui","aquél","aquélla","aquéllas","aquéllos","aquí","arriba","arribaabajo","aseguró","asi","así","atras","aun","aunque","ayer","añadió","aún","b","bajo","bastante","bien","breve","buen","buena","buenas","bueno","buenos","c","cada","casi","cerca","cierta","ciertas","cierto","ciertos","cinco","claro","comentó","como","con","conmigo","conocer","conseguimos","conseguir","considera","consideró","consigo","consigue","consiguen","consigues","contigo","contra","cosas","creo","cual","cuales","cualquier","cuando","cuanta","cuantas","cuanto","cuantos","cuatro","cuenta","cuál","cuáles","cuándo","cuánta","cuántas","cuánto","cuántos","cómo","d","da","dado","dan","dar","de","debajo","debe","deben","debido","decir","dejó","del","delante","demasiado","demás","dentro","deprisa","desde","despacio","despues","después","detras","detrás","dia","dias","dice","dicen","dicho","dieron","diferente","diferentes","dijeron","dijo","dio","donde","dos","durante","día","días","dónde","e","ejemplo","el","ella","ellas","ello","ellos","embargo","empleais","emplean","emplear","empleas","empleo","en","encima","encuentra","enfrente","enseguida","entonces","entre","era","eramos","eran","eras","eres","es","esa","esas","ese","eso","esos","esta","estaba","estaban","estado","estados","estais","estamos","estan","estar","estará","estas","este","esto","estos","estoy","estuvo","está","están","ex","excepto","existe","existen","explicó","expresó","f","fin","final","fue","fuera","fueron","fui","fuimos","g","general","gran","grandes","gueno","h","ha","haber","habia","habla","hablan","habrá","había","habían","hace","haceis","hacemos","hacen","hacer","hacerlo","haces","hacia","haciendo","hago","han","hasta","hay","haya","he","hecho","hemos","hicieron","hizo","horas","hoy","hubo","i","igual","incluso","indicó","informo","informó","intenta","intentais","intentamos","intentan","intentar","intentas","intento","ir","j","junto","k","l","la","lado","largo","las","le","lejos","les","llegó","lleva","llevar","lo","los","luego","lugar","m","mal","manera","manifestó","mas","mayor","me","mediante","medio","mejor","mencionó","menos","menudo","mi","mia","mias","mientras","mio","mios","mis","misma","mismas","mismo","mismos","modo","momento","mucha","muchas","mucho","muchos","muy","más","mí","mía","mías","mío","míos","n","nada","nadie","ni","ninguna","ningunas","ninguno","ningunos","ningún","no","nos","nosotras","nosotros","nuestra","nuestras","nuestro","nuestros","nueva","nuevas","nuevo","nuevos","nunca","o","ocho","os","otra","otras","otro","otros","p","pais","para","parece","parte","partir","pasada","pasado","paìs","peor","pero","pesar","poca","pocas","poco","pocos","podeis","podemos","poder","podria","podriais","podriamos","podrian","podrias","podrá","podrán","podría","podrían","poner","por","porque","posible","primer","primera","primero","primeros","principalmente","pronto","propia","propias","propio","propios","proximo","próximo","próximos","pudo","pueda","puede","pueden","puedo","pues","q","qeu","que","quedó","queremos","quien","quienes","quiere","quiza","quizas","quizá","quizás","quién","quiénes","qué","r","raras","realizado","realizar","realizó","repente","respecto","s","sabe","sabeis","sabemos","saben","saber","sabes","salvo","se","sea","sean","segun","segunda","segundo","según","seis","ser","sera","será","serán","sería","señaló","si","sido","siempre","siendo","siete","sigue","siguiente","sin","sino","sobre","sois","sola","solamente","solas","solo","solos","somos","son","soy","soyos","su","supuesto","sus","suya","suyas","suyo","sé","sí","sólo","t","tal","tambien","también","tampoco","tan","tanto","tarde","te","temprano","tendrá","tendrán","teneis","tenemos","tener","tenga","tengo","tenido","tenía","tercera","ti","tiempo","tiene","tienen","toda","todas","todavia","todavía","todo","todos","total","trabaja","trabajais","trabajamos","trabajan","trabajar","trabajas","trabajo","tras","trata","través","tres","tu","tus","tuvo","tuya","tuyas","tuyo","tuyos","tú","u","ultimo","un","una","unas","uno","unos","usa","usais","usamos","usan","usar","usas","uso","usted","ustedes","v","va","vais","valor","vamos","van","varias","varios","vaya","veces","ver","verdad","verdadera","verdadero","vez","vosotras","vosotros","voy","vuestra","vuestras","vuestro","vuestros","w","x","y","ya","yo","z","él","ésa","ésas","ése","ésos","ésta","éstas","éste","éstos","última","últimas","último","últimos"]

Eliminamos las stopwords de cada documento.

In [5]:
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

Otro preprocesamiento básico que realizaremos es eliminar las palabras que aparecen una sola vez.

In [6]:
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [7]:
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

Actualmente tenemos una lista con listas de los tokens de cada documento.

```
texts = [['portada',
  "'público',",
  'viernes.',
  'fabra',
  'banquillo',
  'supremo;',
  'wikileaks',
  '160',
  'empresas'],
 ['grande!',
  'rt',
  '"el',
  'periodista',
  'alguien',
  'contar',
  'realidad,',
  'vive',
  'ella"',
  'via',
  '@galtares'],
  ...
]
```

Notamos que aún se podrían hacer más preprocesamientos, como eliminar los signos de puntuación, comillas, etc. Para efectos de este notebook no se procesarán más, dado que es un acercamiento simple a lo que permite realizar `gensim`.

Ahora crearemos un diccionario con los textos encontrados.

In [8]:
dictionary = corpora.Dictionary(texts)

Podemos revisar el diccionario que ha sido creado. Veremos los ids que le ha asignado a cada token.

In [9]:
print(dictionary.token2id)

{'portada': 0, "'público',": 1, 'viernes.': 2, 'fabra': 3, 'banquillo': 4, 'supremo;': 5, 'wikileaks': 6, '160': 7, 'empresas': 8, 'grande!': 9, 'rt': 10, '"el': 11, 'periodista': 12, 'alguien': 13, 'contar': 14, 'realidad,': 15, 'vive': 16, 'ella"': 17, 'via': 18, '@galtares': 19, 'gonzalo': 20, 'altozano': 21, 'presentación': 22, 'libro': 23, '101': 24, 'españoles': 25, 'dios.': 26, 'divertido,': 27, 'emocionante': 28, 'brillante.': 29, 'mañana': 30, 'gaceta:': 31, 'tve,': 32, 'pagamos': 33, 'yo,': 34, 'culpa': 35, 'becaria': 36, 'falsa': 37, 'información': 38, 'cierre': 39, '@gaceta': 40, 'envidia': 41, '“@mfcastineiras:': 42, 'pedro': 43, 'paris,': 44, 'esté': 45, 'parisina': 46, 'recordaré': 47, 'gaceta.': 48, 'amaiur': 49, 'depende': 50, 'uxue': 51, 'barkos': 52, 'crear': 53, 'grupo': 54, 'propio.': 55, 'erc': 56, 'cumple': 57, '15%': 58, 'pnv': 59, 'competencia': 60, 'noches': 61, 'followercetes,': 62, 'mitico': 63, 'mi,': 64, 'upyd': 65, 'contará': 66, 'seguro': 67, 'gracias': 